## Supervised Learning using BERT for Sentiment Analysis and Notation Prediction

### Data Extraction and Preprocessing (we follow the same process as in data_cleaning but we keep a different sample of the data)

In [6]:
import pandas as pd
import nltk
from cleantext import clean
import tqdm
import numpy as np
from deep_translator import GoogleTranslator
import torch

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.


In [7]:
torch.cuda.is_available()

True

In [8]:
reviews_df = pd.read_csv("data/reviews.csv", sep=";")
reviews_df.head(3)

,category_link,company_link,company_name,score,description,review_page_nb,review_link,review_score,review_title,review_text,category
0,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a5388a60d6...,5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco
1,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/65a53245a223...,5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco
2,https://fr.trustpilot.com/categories/food_beve...,https://fr.trustpilot.com/review/lefourgon.com,Le Fourgon,"4,9",Le Fourgon vous livre vos boissons consignées ...,329,https://fr.trustpilot.com/reviews/659f0e15dd57...,5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco


In [9]:
reviews_df = reviews_df[["company_name", "score", "review_score", "review_title", "review_text", "category"]]

In [10]:
# we merge the title and the text of the review
reviews_df["review"] = reviews_df["review_title"] + " " + reviews_df["review_text"]
reviews_df.head(3)

,company_name,score,review_score,review_title,review_text,category,review
0,Le Fourgon,"4,9",5,Application conviviale pour passer ses…,Application conviviale pour passer ses command...,food_beverages_tobacco,Application conviviale pour passer ses… Applic...
1,Le Fourgon,"4,9",5,Très facile pour la commande,"Très facile pour la commande, très rapide et l...",food_beverages_tobacco,Très facile pour la commande Très facile pour ...
2,Le Fourgon,"4,9",5,Première expérience réussie !,"Pour nous, c'était une première.Ravis d'avoir ...",food_beverages_tobacco,"Première expérience réussie ! Pour nous, c'éta..."


In [11]:
clean(reviews_df["review"].values[0], no_emoji=True, no_line_breaks=True, lower=False).replace("#", "")

'Application conviviale pour passer ses Application conviviale pour passer ses commandes. Manquent juste les ingredients pas evidents a trouver.Notification par SMS peu de temps avant le creneau choisi pour un creneau precis de 20 minutes.Livreurs agreables.Merci le Fourgon !'

In [12]:
reviews_df["review"] = reviews_df["review"].apply(lambda x: clean(x, no_emoji=True, no_line_breaks=True, lower=False).replace("#", ""))

In [13]:
reviews_df["review"].values[100]

"Exceptionnel La brulerie Belleville est devenue mon unique fournisseur en cafe, le cafe est vraiment d'une qualite exceptionnelle ! De plus, le service apres-vente est d'une qualite humaine devenue bien rare a present !"

#### We Keep only a sample of the data (the top 100 companies with the most reviews) Because of the traduction cost of the "review" column

In [14]:
# number of reviews per company
reviews_df["company_name"].value_counts()[:10]

company_name
Hardloop           236
Alltricks          210
Ekosport           140
vertbaudet         140
Mode Tactique      120
Weenect            120
AAAEP              120
Handball Store     111
Meyclub            100
Bleen              100
Name: count, dtype: int64

In [15]:
# number of reviews in all the dataset
len(reviews_df)

235503

In [16]:
# print the first company name
reviews_df["company_name"].value_counts().index[0]

'Hardloop\xa0'

In [17]:
reviews_df["company_name"] = reviews_df["company_name"].apply(lambda x: x.replace("\xa0", "") if x != "\xa0" else x)
reviews_df["company_name"].value_counts().index[0]

'Hardloop'

In [18]:
# we keep only the reviews of the first 100 companies in terms of number of reviews
revsample_df = reviews_df[reviews_df["company_name"].isin(reviews_df["company_name"].value_counts()[:100].index)]
revsample_df["company_name"].value_counts().sum()

8972

In [19]:
revsample_df["company_name"].value_counts()[:10]

company_name
Hardloop          236
Alltricks         210
vertbaudet        140
Ekosport          140
Weenect           120
Mode Tactique     120
AAAEP             120
Handball Store    111
Plaquedeces.fr    100
JACQUEMUS         100
Name: count, dtype: int64

In [20]:
revsample_df = revsample_df.dropna(ignore_index=True)

In [21]:
# we display all the lines of the first company
pd.set_option("display.max_colwidth", None)
revsample_df[revsample_df["company_name"] == "Hardloop"]

,company_name,score,review_score,review_title,review_text,category,review
7658,Hardloop,"4,5",1,Première commande sur ce site,"Première commande sur ce site, et extrêmement déçue !Pantalon de randonnée commandé pour un cadeau de Noël début décembre, il n’est toujours pas réceptionné malgré les dates indiquées sur le site. Diverses relances auprès du service client par e-mail et Instagram afin d’obtenir à minima un retour sur la livraison ou sur un remboursement éventuel pour l’acheter ailleurs : aucun retour.Une honte.À fuir…Merci de supprimer mon adresse e-mail privée à ce commentaire (méthode très douteuse pour que je supprime mon commentaire négatif).",sports,"Premiere commande sur ce site Premiere commande sur ce site, et extremement decue !Pantalon de randonnee commande pour un cadeau de Noel debut decembre, il n'est toujours pas receptionne malgre les dates indiquees sur le site. Diverses relances aupres du service client par e-mail et Instagram afin d'obtenir a minima un retour sur la livraison ou sur un remboursement eventuel pour l'acheter ailleurs : aucun retour.Une honte.A fuirMerci de supprimer mon adresse e-mail privee a ce commentaire (methode tres douteuse pour que je supprime mon commentaire negatif)."
7659,Hardloop,"4,5",1,Pantalon à 150€ reçu 10 jours après la…,"Pantalon à 150€ reçu 10 jours après la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site émet systématiquement un message d'erreur quand on en fait la demande. Le numéro de tel du prétendu standard n'est en fait qu'un simple répondeur, où il n'est même pas possible de laisser un message.Finalement, après deux mails envoyés, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement reçu un mois après l'envoi... mais reçu immédiatement après après que j'aie renvoyé un mailde reclamation. À fuir.",sports,"Pantalon a 150 recu 10 jours apres la Pantalon a 150 recu 10 jours apres la commande. Pour cause de taille trop grande, impossible d'obtenir un bon de retour pour retourner l'article, car le site emet systematiquement un message d'erreur quand on en fait la demande. Le numero de tel du pretendu standard n'est en fait qu'un simple repondeur, ou il n'est meme pas possible de laisser un message.Finalement, apres deux mails envoyes, j'ai fini par recevoir un bon de retour imprimable. Pas de remboursement recu un mois apres l'envoi... mais recu immediatement apres apres que j'aie renvoye un mailde reclamation. A fuir."
7660,Hardloop,"4,5",5,Equipe très réactive en cas de problème…,Equipe très réactive en cas de problème sur une commande pour trouver des solutions.,sports,Equipe tres reactive en cas de probleme Equipe tres reactive en cas de probleme sur une commande pour trouver des solutions.
7661,Hardloop,"4,5",4,Très bon service,"Beaucoup de choix, produits bien emballés, et livraison rapide.",sports,"Tres bon service Beaucoup de choix, produits bien emballes, et livraison rapide."
7662,Hardloop,"4,5",5,Parfait,"Parfait, livraison rapide et service optimal, je recommande",sports,"Parfait Parfait, livraison rapide et service optimal, je recommande"
...,...,...,...,...,...,...,...
8676,Hardloop,"4,0",5,"Competitive price, rapid delivery","Competitive price, rapid delivery to Poland",vehicles_transportation,"Competitive price, rapid delivery Competitive price, rapid delivery to Poland"
8677,Hardloop,"4,0",1,Szczytem chamstwa nie jest dostarczyć…,"Szczytem chamstwa nie jest dostarczyć produkt, który zgodnie z prawem zakupiłem, i który jest dostępny w sklepie, tylko dlatego, że właścicielowi nie podoba się kwota, za którą został sprzedany. W systemie była widoczna cena w promocji. I z niej korzystając kupiłem kurtkę Helly Hannsena. Anulowanie mojego kupna i zwrot pieniędzy zasłaniając się rzekomym błędem w wyświetlaniu ceny jest co najmniej nie na miejscu. Jeżeli nawet do takiego doszło właściciel powinien rozwiązywać tą kwestię, wśród swoich pracowników odpowiedzialnych za to

We keep only the reviews in french

In [22]:
from nltk import wordpunct_tokenize
from nltk.corpus import stopwords
print(stopwords.fileids())

['arabic', 'azerbaijani', 'basque', 'bengali', 'catalan', 'chinese', 'danish', 'dutch', 'english', 'finnish', 'french', 'german', 'greek', 'hebrew', 'hinglish', 'hungarian', 'indonesian', 'italian', 'kazakh', 'nepali', 'norwegian', 'portuguese', 'romanian', 'russian', 'slovene', 'spanish', 'swedish', 'tajik', 'turkish']


In [23]:
def detecte_langage(message):
    # on definie un dictionnaire vide
    # {langue : nbre de stopwords communs 
    # entre langue et les mots de message}
    languages_shared_words = {}
    # tokenization en mots
    words = wordpunct_tokenize(message)
    for language in stopwords.fileids():
        # stopwords pour chaque langue
        stopwords_liste = stopwords.words(language)
        # on retire les doublons
        words = set(words)
        # les mots communs entre stopwords 
        # d'une langue et les mots de message
        common_elements = words.intersection(stopwords_liste)
        # ajout du couple au dictionnaire
        languages_shared_words[language] = len(common_elements)
    # on retourne la langue avec le max de mots commun
    return  max(languages_shared_words, key = languages_shared_words.get)

In [24]:
# on applique la fonction pour chaque message sur la colonne review et on ne garde que les lignes ou la langue est en francais
revsample_df["langue"] = revsample_df["review"].apply(detecte_langage)

In [25]:
revsample_df[revsample_df["langue"] == "french"].count()

company_name    7729
score           7729
review_score    7729
review_title    7729
review_text     7729
category        7729
review          7729
langue          7729
dtype: int64

In [26]:
revsample_df_fr = revsample_df[revsample_df["langue"] == "french"]

Training CamemBERT : 

In [27]:
from transformers import CamembertTokenizer

In [28]:
# let's use Cambembert model for French
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [29]:
# # we tokenize the reviews
# revsample_df_fr["review"] = revsample_df_fr["review"].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# revsample_df_fr["review"].values[0]

In [30]:
# we split the dataset into train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(revsample_df_fr, test_size=0.2, random_state=42)

train.shape, test.shape

((6183, 8), (1546, 8))

In [31]:
train.head(3)

,company_name,score,review_score,review_title,review_text,category,review,langue
1638,Plaquedeces.fr,"4,6",5,Plaque réalisé très vite et très bien livraison rapide et bonne condition,Plaque commandée un vendredi tard dans la soirée depuis le Québec pour une distribution le mardi en région Parisienne pour des funérailles le mercredi. Nous avons utilisé la livraison express. La plaque est arrivée en très bonne condition,construction_manufactoring,Plaque realise tres vite et tres bien livraison rapide et bonne condition Plaque commandee un vendredi tard dans la soiree depuis le Quebec pour une distribution le mardi en region Parisienne pour des funerailles le mercredi. Nous avons utilise la livraison express. La plaque est arrivee en tres bonne condition,french
2598,Recharge.fr,"4,3",5,Site fiable,"J'ai fait plusieurs rechargement en passant par ce site ""Recharge.fr"" et j'en suis totalement content, cela est rapide et payable en plus par PayPal, c'est vraiment tout bénéfique, les chiffres donné pour pouvoir recharger fonctionne très bien franchement rien a redire, parfait",events_entertainment,"Site fiable J'ai fait plusieurs rechargement en passant par ce site ""Recharge.fr"" et j'en suis totalement content, cela est rapide et payable en plus par PayPal, c'est vraiment tout benefique, les chiffres donne pour pouvoir recharger fonctionne tres bien franchement rien a redire, parfait",french
5612,Taskrabbit France,"4,5",5,Service impeccable,"J'ai acheté un meuble chez Ikea et j'ai demandé un service de montage à domicile. L'entreprise Taskrabbit m'a contactée et le ""taskeur"" qui est venu chez moi a vraiment été très efficace. Le meuble a été monté de manière rapide et efficace. Le montant payé est dérisoire face à la prestation réalisée. Je recommande ++",home_services,"Service impeccable J'ai achete un meuble chez Ikea et j'ai demande un service de montage a domicile. L'entreprise Taskrabbit m'a contactee et le ""taskeur"" qui est venu chez moi a vraiment ete tres efficace. Le meuble a ete monte de maniere rapide et efficace. Le montant paye est derisoire face a la prestation realisee. Je recommande ++",french


In [32]:
# function to encode the reviews with a max_length of 512
max_length = 512
def encode_reviews(tokenizer, reviews, max_length):
    token_ids = np.zeros(shape=(len(reviews), max_length),
                         dtype=np.int32)
    for i, review in enumerate(reviews):
        encoded = tokenizer.encode(review, max_length=max_length)
        token_ids[i, 0:len(encoded)] = encoded
    attention_mask = (token_ids != 0).astype(np.int32)
    return {"input_ids": token_ids, "attention_mask": attention_mask}

In [33]:
# we encode the reviews
train_encoded = encode_reviews(tokenizer, train["review"].values, max_length)
test_encoded = encode_reviews(tokenizer, test["review"].values, max_length)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [34]:
train_encoded

{'input_ids': array([[    5, 21921,   343, ...,     0,     0,     0],
        [    5,  1082,  5758, ...,     0,     0,     0],
        [    5,  2953, 10931, ...,     0,     0,     0],
        ...,
        [    5,    74,  4324, ...,     0,     0,     0],
        [    5,  9519,    22, ...,     0,     0,     0],
        [    5,  1082, 14951, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [41]:
# creation of train and test labels
train_labels = train["review_score"].values
test_labels = test["review_score"].values

train_labels

array([5, 5, 5, ..., 5, 5, 5], dtype=int64)

In [42]:
import tensorflow as tf

In [43]:
from transformers import TFCamembertForSequenceClassification

In [44]:
# now, we train the Camembert model
# loading the Camembert model
camembert_model = TFCamembertForSequenceClassification.from_pretrained("camembert-base")

# defining the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

# defining the loss function
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

All PyTorch model weights were used when initializing TFCamembertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFCamembertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [45]:
# we compile the model
camembert_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

# we train the model
camembert_model.fit(train_encoded, train_labels, epochs=3, batch_size=16)

Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 196, in _run_module_as_main

  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 86, in _run_code

  File "c:\Users\gdetr\Documents\ESILV A5\Machine Learning for NLP\NLP_Project\myenv\lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "c:\Users\gdetr\Documents\ESILV A5\Machine Learning for NLP\NLP_Project\myenv\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance

  File "c:\Users\gdetr\Documents\ESILV A5\Machine Learning for NLP\NLP_Project\myenv\lib\site-packages\ipykernel\kernelapp.py", line 739, in start

  File "c:\Users\gdetr\Documents\ES

In [ ]:
# we evaluate the model
camembert_model.evaluate(test_encoded, test_labels)

# we predict the test labels
test_pred = camembert_model.predict(test_encoded)

test_pred